In [50]:
import os
from dotenv import load_dotenv

import bs4
import tiktoken
import numpy as np
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate


In [9]:
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")


In [ ]:
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

In [12]:
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)

In [14]:
retriever = vectorstore.as_retriever()

In [15]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

In [17]:
# Assuming you have already executed the code that builds the vectorstore (from previous steps)

user_question = "where did harrison work?"

# Use the retriever object to query the vector store
retrieved_info = retriever.invoke(user_question)

# Print the retrieved information (might be an embedding or similar)
print(retrieved_info)

[Document(page_content='harrison worked at kensho')]


In [18]:
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

In [19]:
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [20]:
user_question = "where did harrison work?"
output = retrieval_chain.invoke(user_question)
print(output)

Harrison worked at Kensho.


In [21]:
text = """
Imagine you're working on a team project where you need to pass information around between different classmates and tools. RunnablePassthrough from LangChain is like a special box you can use to hand information to the next person without changing it.

Think of information as instructions or questions. In your project, these could be things like "write a poem" or "find information about the solar system."
The box is RunnablePassthrough. You put the instructions or question in the box, and it gets delivered to the next classmate (tool) exactly as it is.
Why use this box? Here are two main reasons:

Keeping things clear: Sometimes you just want to pass information along exactly as written. The passthrough box helps avoid accidentally adding or changing anything. It's like making sure your instructions to your classmates are clear and easy to follow.
Working with different tools: Different classmates might need information in a specific way. The passthrough box ensures the information stays in the right format for the next step. It's like using different folders to organize notes for different parts of your project.
Example: Combining a Research Assistant with a Summarizer:

Let's say you have a classmate really good at finding information online (like a search engine).
You have another classmate who's super good at summarizing things (like a Summarization LLM).
You want to know "what are the effects of climate change?"
You can use the research assistant to find relevant articles, then use the passthrough box to deliver those articles (unchanged) to the summarizer classmate.
The summarizer can then use both the original question and the articles to create a summary of the effects of climate change.
RunnablePassthrough helps projects run smoothly by ensuring information is passed clearly and in the right format, just like using clear communication and organization helps your team projects succeed!
"""

In [27]:
vectorstore = FAISS.from_texts(
    [text], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever(kwargs=2)

In [28]:
# Assuming you have already executed the code that builds the vectorstore (from previous steps)

user_question = "what is RunnablePassthrough in langchain"

# Use the retriever object to query the vector store
retrieved_info = retriever.invoke(user_question)

# Print the retrieved information (might be an embedding or similar)
print(retrieved_info)

[Document(page_content='\nImagine you\'re working on a team project where you need to pass information around between different classmates and tools. RunnablePassthrough from LangChain is like a special box you can use to hand information to the next person without changing it.\n\nThink of information as instructions or questions. In your project, these could be things like "write a poem" or "find information about the solar system."\nThe box is RunnablePassthrough. You put the instructions or question in the box, and it gets delivered to the next classmate (tool) exactly as it is.\nWhy use this box? Here are two main reasons:\n\nKeeping things clear: Sometimes you just want to pass information along exactly as written. The passthrough box helps avoid accidentally adding or changing anything. It\'s like making sure your instructions to your classmates are clear and easy to follow.\nWorking with different tools: Different classmates might need information in a specific way. The passthro

In [29]:

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("what is RunnablePassthrough in langchain")

'RunnablePassthrough in LangChain is a special box that allows for the passing of information between different classmates and tools without changing it, ensuring that the information stays in the right format for the next step of the project.'

In [33]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

In [36]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)


In [39]:
splits[:15]

[Document(page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n

In [40]:
vectorstore = FAISS.from_documents(
    documents=splits[0:15],
    embedding=OpenAIEmbeddings()
)

In [41]:
retriever = vectorstore.as_retriever()

In [42]:
"""
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question} 

Context: {context} 

Answer:
    """
prompt = hub.pull("rlm/rag-prompt")


# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


In [43]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [44]:
# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is a technique used to break down complex tasks into smaller and simpler steps, making it easier for autonomous agents to plan and execute them effectively. It involves transforming big tasks into multiple manageable tasks by thinking step by step or exploring multiple reasoning possibilities at each step. Task decomposition can be done using simple prompting, task-specific instructions, or by relying on an external classical planner for long-horizon planning.'

In [65]:
question = "what kinds of pets do I like?"
document = "My favorite pet is a cat."

In [57]:
def num_tokens_from_string(string:str, encoding_name:str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

8

In [63]:
embd= OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
print(len(query_result))
print(len(document_result))

1536
1536


In [64]:
def cosin_similarity(vec1,vec2):
    dot_product = np.dot(vec1,vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1*norm_vec2)

similarity = cosin_similarity(query_result,document_result)
print(f"Cosine Similarity : {similarity}")

Cosine Similarity : 0.7193982605515085


In [66]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

In [67]:
# Split

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

In [68]:
vectorstore = Chroma.from_documents(documents=splits[0:15], 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [70]:
retriever = vectorstore.as_retriever(search_kwargs={"k":1})
docs= retriever.get_relevant_documents("what is task decomposition?")

print(docs)
print(len(docs))

[Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via 

In [71]:
# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])

In [72]:
chain = prompt | llm

chain.invoke({"context":docs, "question":"what is task decomposition?"})

AIMessage(content='Task decomposition is the process of breaking down a complex task into smaller and simpler steps in order to make it more manageable for an autonomous agent system. This can be achieved through techniques like Chain of Thought and Tree of Thoughts, which prompt the agent to think step by step and explore multiple reasoning possibilities at each step. Task decomposition can also be done using simple prompting, task-specific instructions, or human inputs.', response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 330, 'total_tokens': 410}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a0c9b83b-7fc3-42d9-aaaf-85330a4b6461-0')